In [1]:
%matplotlib inline
import os, sys
sys.path.append('/Users/hhsieh/anaconda3/envs/astroconda/lib/python3.6/site-packages')
sys.path.append('/opt/local/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages')
print(sys.version)
print('')
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.io.fits import getheader
from astropy.modeling import models
from astropy import nddata
from astropy import units as u
import ccdproc as cp
from ccdproc import CCDData, ImageFileCollection
import numpy as np
import glob, os, bz2, subprocess
import sqlite3
from sqlite3 import Error
import datetime


3.6.11 | packaged by conda-forge | (default, Aug  5 2020, 20:19:23) 
[GCC Clang 10.0.1 ]



In [2]:
def create_directory(path):
    if not os.path.exists(path):
        os.mkdir(path)
        print('Directory {:s} successfully created.'.format(path))
    else:
        print('Directory {:s} already exists.'.format(path))
    return None

def decompress_file_bzip2(filename):
    cmd = ['bzip2','-d',filename]
    process = subprocess.call(cmd)
    return None

def compress_file_fpack(filename):
    fpack_command  = '/Users/hhsieh/Astro/tools/cfitsio/fpack'
    delete_command = '/bin/rm'
    fpack_cmd = [fpack_command,filename]
    process = subprocess.call(fpack_cmd)
    delete_cmd = [delete_command,filename]
    process = subprocess.call(delete_cmd)
    return None

def decompress_file_fpack(filename):
    funpack_command = '/Users/hhsieh/Astro/tools/cfitsio/funpack'
    delete_command  = '/bin/rm'
    funpack_cmd = [funpack_command,filename]
    process = subprocess.call(funpack_cmd)
    delete_cmd = [delete_command,filename]
    process = subprocess.call(delete_cmd)
    return None

def decompress_directory_bzip2(file_path):
    print('>>> Starting decompression of bz2 files in {:s}...'.format(file_path))
    os.chdir(file_path)
    for bz2_file in glob.glob('*.bz2'):
        decompress_file_bzip2(bz2_file)
        #cmd = [bzip1,bzip2,bz2_file]
        #process = subprocess.call(cmd)
    print('>>> Decompression of bz2 files complete.')
    return None

def compress_directory_fpack(file_path):
    print('>>> Starting fpack compression of FITS files in {:s}...'.format(file_path))
    #fpack_file = '/Users/hhsieh/Astro/tools/cfitsio/fpack'
    #delete_file = '/bin/rm'
    os.chdir(file_path)
    for fits_file in glob.glob('*.fits'):
        compress_file_fpack(fits_file)
        #fpack_cmd = [fpack_file,fits_file]
        #process = subprocess.call(fpack_cmd)
        #delete_cmd = [delete_file,fits_file]
        #process = subprocess.call(delete_cmd)
    print('>>> fpack compression of FITS files complete.')
    return None
    
def decompress_directory_fpack(file_path):
    print('>>> Starting decompression of fz files in {:s}...'.format(file_path))
    #funpack_file = '/Users/hhsieh/Astro/tools/cfitsio/funpack'
    #delete_file = '/bin/rm'
    os.chdir(file_path)
    for fz_file in glob.glob('*.fz'):
        decompress_file_fpack(fz_file)
        #funpack_cmd = [funpack_file,fz_file]
        #process = subprocess.call(funpack_cmd)
        #delete_cmd = [delete_file,fz_file]
        #process = subprocess.call(delete_cmd)
    print('>>> Decompression of fz files complete.')
    return None


#def decompress_data_bzip2():
#    print('>>> Starting decompression of bz2 files...')
#    bzip1 = 'bzip2'
#    bzip2 = '-d'
#    for bz2_file in glob.glob('*.bz2'):
#        cmd = [bzip1,bzip2,bz2_file]
#        process = subprocess.call(cmd)
#    print('>>> Decompression of bz2 files complete.')
#
#def decompress_data_fpack():
#    print('>>> Starting decompression of fz files...')
#    funpack_file = '/Users/hhsieh/Astro/tools/cfitsio/funpack'
#    delete_file = '/bin/rm'
#    for fz_file in glob.glob('*.fz'):
#        funpack_cmd = [funpack_file,fz_file]
#        process = subprocess.call(funpack_cmd)
#        delete_cmd = [delete_file,fz_file]
#        process = subprocess.call(delete_cmd)
#    print('>>> Decompression of fz files complete.')
#
#def compress_data_fpack():
#    print('>>> Starting fpack compression of fits files...')
#    fpack_file = '/Users/hhsieh/Astro/tools/cfitsio/fpack'
#    delete_file = '/bin/rm'
#    for fits_file in glob.glob('*.fits'):
#        fpack_cmd = [fpack_file,fits_file]
#        process = subprocess.call(fpack_cmd)
#        delete_cmd = [delete_file,fits_file]
#        process = subprocess.call(delete_cmd)
#    print('>>> Compression of fits files complete.')

def get_date_id():
    # obtain YYYYMMDD date from filename of GMOS FITS files being processed
    # return: YYYYMMDD date id
    for raw_mef_file in glob.glob('*.fits'):
        fits_date = raw_mef_file[1:9]
    return fits_date

def create_stats_files(fits_date):
    for extid in range(0,6):
        ext_stats = 'n' + fits_date + '.{:02d}.stats'.format(extid+1)
        outputfile = open(ext_stats,'w')
        outputfile.write('# Extension {:02d}               NPIX        MEAN     STDDEV         MIN         MAX\n'.format(extid+1))
        outputfile.close()
    return None
        
def compile_stats_files(fits_date,outputfile_path):
    print('Starting image statistics output...')
    output_stats_filename = outputfile_path + 'n' + fits_date + '.stats'
    output_stats_file = open(output_stats_filename,'w')
    cmd1 = '/bin/rm'
    for stats_filename in glob.glob('*.??.stats'):
        stats_file = open(stats_filename,'r')
        for line in stats_file:
            output_stats_file.write(line)
        stats_file.close()
        cmd = [cmd1,stats_filename]
        process = subprocess.call(cmd)
    output_stats_file.close()
    print('>>> Image statistics output complete.')
    return None

def remove_test_images():
    # remove test images from downloaded Gemini data tarball
    delete_cmd = '/bin/rm'
    for fits_file in glob.glob('*.fits'):
        hdulist = fits.open(fits_file)
        hdr = hdulist[1].header
        naxis2 = hdr['naxis2']
        if(naxis2 < 200):
            delete_file = [delete_cmd,fits_file]
            print('Removing {:s}...'.format(fits_file))
            process = subprocess.call(delete_file)
    return None

def split_extensions():
    # split 6-element GMOS multi-extension FITS files into individual elements
    # return: dimensions of extensions; also writes individual extension files to working directory
    for raw_mef_file in glob.glob('*.fits'):
        hdulist = fits.open(raw_mef_file)
        print('Splitting {:s}...'.format(raw_mef_file))
        fits_date = raw_mef_file[1:9]
        fits_id   = raw_mef_file[10:14]
        hdr1 = getheader(raw_mef_file,0)
        for extid in range(0,6):
            extension = hdulist[extid+1].data
            hdr2 = hdulist[extid+1].header
            radecsys = hdr2['RADECSYS']
            hdr2['RADESYSA'] = radecsys
            del hdr2['RADECSYS']
            imstat_npix = extension.size
            imstat_min = np.min(extension)
            imstat_max = np.max(extension)
            imstat_mean = np.mean(extension)
            imstat_stdev = np.std(extension)
            ext_filename = 'n' + fits_date + '.' + fits_id + '.{:02d}.fits'.format(extid+1)
            outputfilename = 'n' + fits_date + '.{:02d}.stats'.format(extid+1)
            outputfile = open(outputfilename,'a')
            outputfile.write('{:s}   {:>8d}    {:>8.2f}    {:>7.2f}    {:>8.2f}    {:>8.2f}\n'.format(ext_filename,imstat_npix,imstat_mean,imstat_stdev,imstat_min,imstat_max))
            outputfile.close()
            dimensions = extension.shape
            dimension1 = dimensions[0]
            dimension2 = dimensions[1]
            hdr = hdr1 + hdr2
            fits.writeto(ext_filename,extension,hdr)
        hdulist.close()
    return dimension1, dimension2
    print('{:s} - >>> Multi-extension fits file splitting complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    return None
    
def remove_wrong_dimensions(dimension1,dimension2):
    # identify bias files with correct binning and remove others
    # return: none; retains bias files with correct binning and removes others
    for raw_mef_file in glob.glob('*.fits'):
        hdulist = fits.open(raw_mef_file)
        extension01 = hdulist[1].data
        bias_dimensions = extension01.shape
        bias_dimension1 = bias_dimensions[0]
        bias_dimension2 = bias_dimensions[1]
        hdulist.close()
        delete_file = '/bin/rm'
        if (bias_dimension1 != dimension1) or (bias_dimension2 != dimension2):
            delete_cmd = [delete_file,raw_mef_file]
            process = subprocess.call(delete_cmd)
    print('{:s} - >>> Bias frame selection complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    return None

def overscan_and_trim_gmosn():
    # Do overscan correction and trim images in current directory
    print('Starting overscan correction and trimming...')
    for fits_file in glob.glob('*.????.??.fits'):
        ot_fits_file = fits_file[0:17] + '.ot.fits'
        fits_data = CCDData.read(fits_file,unit=u.adu)
        file_ext = fits_file[15:17]
        # Overscan correction
        if ((file_ext == '01') or (file_ext == '03')):
            o_fits_data  = cp.subtract_overscan(fits_data, fits_section='[1:32,1:2304]', overscan_axis=1, add_keyword={'oscansub': True, 'calstat': 'O'}, model=models.Polynomial1D(1))
        else: # if file_ext == 4 or 5
            o_fits_data  = cp.subtract_overscan(fits_data, fits_section='[513:544,1:2304]', overscan_axis=1, add_keyword={'oscansub': True, 'calstat': 'O'}, model=models.Polynomial1D(1))
        # Image trimming for Ext #s 1,3,4,5
        if ((file_ext == '01') or (file_ext == '03')):
            ot_fits_data = cp.trim_image(o_fits_data,fits_section='[33:543,3:2260]',add_keyword={'trimmed': True, 'calstat': 'OT'})
            ot_fits_data.write(ot_fits_file)
        elif ((file_ext == '04') or (file_ext == '05')):
            ot_fits_data = cp.trim_image(o_fits_data,fits_section='[2:512,3:2260]',add_keyword={'trimmed': True, 'calstat': 'OT'})
            ot_fits_data.write(ot_fits_file)
        delete_file = '/bin/rm'
        delete_cmd = [delete_file,fits_file]
        process = subprocess.call(delete_cmd)
    print('{:s} - >>> Overscan correction and trimming complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    return None

def bias_median_combine(dateid):
    print('Starting median combination of bias frames...')
    for extid in range(0,6):
        if((extid == 0) or (extid == 2) or (extid == 3) or (extid == 4)):
            bias_list = []
            ext_file_format = '*.{:02d}.ot.fits'.format(extid+1)
            output_filename = 'n' + dateid + '.bias.{:02d}.fits'.format(extid+1)
            for fits_file in glob.glob(ext_file_format):
                fits_data = CCDData.read(fits_file)
                bias_list.append(fits_data)
                delete_file = '/bin/rm'
                delete_cmd = [delete_file,fits_file]
                process = subprocess.call(delete_cmd)
            master_bias = cp.combine(bias_list,method='median')
            master_bias.write(output_filename)
    print('{:s} - >>> Median combination of bias frames complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    return None

def bias_correct(dateid,cwd_raw_bias):
    print('Starting bias subtraction...')
    delete_cmd = '/bin/rm'
    for extid in range(0,6):
        if((extid == 0) or (extid == 2) or (extid == 3) or (extid == 4)):
            bias_filename = cwd_raw_bias + 'n' + dateid + '.bias.{:02d}.fits'.format(extid+1)
            ext_file_format = '*.{:02d}.ot.fits'.format(extid+1)
            for fits_file in glob.glob(ext_file_format):
                fits_data = CCDData.read(fits_file)
                bias_data = CCDData.read(bias_filename)
                fits_date_imageid = fits_file[0:14]
                output_filename = fits_date_imageid + '.{:02d}.otz.fits'.format(extid+1)
                bias_corrected_data = cp.subtract_bias(fits_data,bias_data,add_keyword={'zerocorr': True, 'calstat': 'OTZ'})
                bias_corrected_data.write(output_filename)
                delete_file = [delete_cmd,fits_file]
                process = subprocess.call(delete_file)
    print('{:s} - >>> Bias subtraction complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    return None

def concatenate_gmos_amps():
    print('>>> Starting adjacent amp area concatenation...')
    delete_cmd = '/bin/rm'
    move_cmd = 'mv'
    
    for fits_file in glob.glob('*.01.otz.fits'):
        file_prefix = fits_file[0:14]
        output_filename = file_prefix + '.chip1.otz.fits'
        move_file = [move_cmd,fits_file,output_filename]
        process = subprocess.call(move_file)

    for fits_file in glob.glob('*.03.otz.fits'):
        file_prefix = fits_file[0:14]
        ext1_filename = file_prefix + '.04.otz.fits'
        ext2_filename = file_prefix + '.03.otz.fits'
        hdr = getheader(ext1_filename,0)
        hdulist1 = fits.open(ext1_filename)
        ext1_data = hdulist1[0].data
        hdulist2 = fits.open(ext2_filename)
        ext2_data = hdulist2[0].data
        hdulist1.close()
        hdulist2.close()
        chip1_data = np.hstack((ext1_data,ext2_data))
        output_filename = file_prefix + '.chip2.otz.fits'
        fits.writeto(output_filename,chip1_data,hdr)
        delete_file = [delete_cmd,ext1_filename]
        process = subprocess.call(delete_file)
        delete_file = [delete_cmd,ext2_filename]
        process = subprocess.call(delete_file)
        
    for fits_file in glob.glob('*.05.otz.fits'):
        file_prefix = fits_file[0:14]
        output_filename = file_prefix + '.chip3.otz.fits'
        move_file = [move_cmd,fits_file,output_filename]
        process = subprocess.call(move_file)
        
    print('{:s} - >>> Adjacent amp area concatenation complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    return None


def process_flatfield_files(path_rawfits,path_rawbias,path_procfits,instr_id):
    
    print('\n')
    create_directory(path_procfits)
    
    print('\n{:s} - Decompressing and splitting data files...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    os.chdir(path_rawfits)
    decompress_directory_bzip2(path_rawfits)      #decompress downloaded GMOS files
    decompress_directory_fpack(path_rawfits)      #decompress previously fpacked GMOS files
    remove_test_images()
    date_id = get_date_id()
    create_stats_files(date_id)                   #create files to record image statistics of bias files
    dim1,dim2 = split_extensions()                #split science MEFs into individual FITS files
    compile_stats_files(date_id,path_procfits)    #collect image statistics of science files together and delete individual files

    print('\n{:s} - Decompressing, filtering, and splitting bias files...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    os.chdir(path_rawbias)
    decompress_directory_bzip2(path_rawbias)      #decompress downloaded GMOS files
    decompress_directory_fpack(path_rawbias)      #decompress previously fpacked GMOS files
    remove_test_images()
    remove_wrong_dimensions(dim1,dim2)            #find and retain bias frames matching data file binning
    create_stats_files(date_id)                   #create files to record image statistics of bias files
    split_extensions()                            #split bias MEFs into individual FITS files
    compile_stats_files(date_id+'.bias',path_procfits)          #collect image statistics of bias files together and delete individual files
        
    # perform overscan corrections and trim data for bias frames
    os.chdir(path_rawbias)
    if instr_id == 'gmosn':
        overscan_and_trim_gmosn()
    if instr_id == 'gmoss':
        overscan_and_trim_gmoss()
    bias_median_combine(date_id)
        
    # perform overscan corrections and trim data for science frames
    os.chdir(path_rawfits)
    if instr_id == 'gmosn':
        overscan_and_trim_gmosn()
    if instr_id == 'gmoss':
        overscan_and_trim_gmoss()
    bias_correct(date_id,path_rawbias)

    # join science frames
    os.chdir(path_rawfits)
    concatenate_gmos_amps()
    
    # move processed files to processed directory
    os.chdir(path_rawfits)
    for fits_file in glob.glob('*.chip?.otz.fits'):
        cmd_mv = 'mv'
        move_file_to_flat_dir = [cmd_mv,fits_file,path_procfits]
        process = subprocess.call(move_file_to_flat_dir)
    os.chdir(path_rawbias)
    for fits_file in glob.glob('*.bias.??.fits'):
        cmd_mv = 'mv'
        move_file_to_flat_dir = [cmd_mv,fits_file,path_procfits]
        process = subprocess.call(move_file_to_flat_dir)

    compress_directory_fpack(path_rawfits)
    compress_directory_fpack(path_rawbias)
    compress_directory_fpack(path_procfits)
        
    return None
        
print('Done.')


Done.


In [3]:
def process_flatfield_directory(basewd):
    os.chdir(basewd)
    for datadir in glob.glob('ut*_gemini?_twiskyflat'):
        path_rawfits  = basewd + datadir + '/rawfits_twiskyflat/'
        path_rawbias  = basewd + datadir + '/rawfits_bias/'
        path_procfits = basewd + datadir + '/procfits_twiskyflat/'
    
        instr_id = 'not recognized'
        if datadir[-12:-11] == 'N': instr_id = 'gmosn'
        if datadir[-12:-11] == 'S': instr_id = 'gmoss'
    
        if not os.path.exists(path_procfits):
            if os.path.exists(path_rawfits):
                if len(os.listdir(path_rawfits)) > 0:
                    if os.path.exists(path_rawbias):
                        if len(os.listdir(path_rawbias)) > 0:
                            if instr_id != 'not recognized':
                                if instr_id == 'gmosn':
                                    process_flatfield_files(path_rawfits,path_rawbias,path_procfits,instr_id)
                                else:
                                    print('{:s} - GMOS-S reduction code not yet available.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
                            else:
                                print('{:s} - Instrument for {:s} not recognized.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
                        else:
                            print('{:s} - Bias directory in {:s} is empty.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
                    else:
                        print('{:s} - Bias directory in {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
                else:
                    print('{:s} - Flatfield directory in {:s} is empty.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
            else:
                print('{:s} - Flatfield directory in {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
        else:
            print('{:s} - Data in {:s} already processed.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
    return None

# Set base working directory and subdirectory paths

#basewd = '/volumes/Fantom12a/BackupData/gemini/flatfields_twilight/GMOS-N_TwilightFlats_20111015_20170202/GMOS-N_Twilight_Flats_g/'
#basewd = '/volumes/Fantom12a/BackupData/gemini/flatfields_twilight/GMOS-N_TwilightFlats_20111015_20170202/GMOS-N_Twilight_Flats_r/'
#basewd = '/volumes/Fantom12a/BackupData/gemini/flatfields_twilight/GMOS-N_TwilightFlats_20111015_20170202/GMOS-N_Twilight_Flats_i/'
#basewd = '/volumes/Fantom12a/BackupData/gemini/flatfields_twilight/GMOS-S_TwilightFlats_20140601_20170301/GMOS-S_Twilight_Flats_g/'
#basewd = '/volumes/Fantom12a/BackupData/gemini/flatfields_twilight/GMOS-S_TwilightFlats_20140601_20170301/GMOS-S_Twilight_Flats_r/'
#basewd = '/volumes/Fantom12a/BackupData/gemini/flatfields_twilight/GMOS-S_TwilightFlats_20140601_20170301/GMOS-S_Twilight_Flats_i/'
#rawfits_dir   = '/rawfits_twiskyflat/'
#rawbias_dir   = '/rawfits_bias/'
#procfits_dir  = '/procfits_twiskyflat/'


In [4]:
#process_flatfield_directory('/volumes/Fantom12a/BackupData/gemini/flatfields_twilight/GMOS-N_TwilightFlats_20111015_20170202/GMOS-N_Twilight_Flats_g/')
#process_flatfield_directory('/volumes/Fantom12a/BackupData/gemini/flatfields_twilight/GMOS-N_TwilightFlats_20111015_20170202/GMOS-N_Twilight_Flats_r/')
#process_flatfield_directory('/volumes/Fantom12a/BackupData/gemini/flatfields_twilight/GMOS-N_TwilightFlats_20111015_20170202/GMOS-N_Twilight_Flats_i/')
##process_flatfield_directory('/volumes/Fantom12a/BackupData/gemini/flatfields_twilight/GMOS-S_TwilightFlats_20140601_20150101/GMOS-S_Twilight_Flats_g/')
##process_flatfield_directory('/volumes/Fantom12a/BackupData/gemini/flatfields_twilight/GMOS-S_TwilightFlats_20140601_20150101/GMOS-S_Twilight_Flats_r/')
##process_flatfield_directory('/volumes/Fantom12a/BackupData/gemini/flatfields_twilight/GMOS-S_TwilightFlats_20140601_20150101/GMOS-S_Twilight_Flats_i/')

process_flatfield_directory('/Users/hhsieh/Data/176P/ut20051229_gemini2_twiskyflat/')

#os.chdir(basewd)
#for datadir in glob.glob('ut*_gemini?_twiskyflat'):
#    path_rawfits  = basewd + datadir + rawfits_dir
#    path_rawbias  = basewd + datadir + rawbias_dir
#    path_procfits = basewd + datadir + procfits_dir
#    
#    instr_id = 'not recognized'
#    if datadir[-12:-11] == 'N': instr_id = 'gmosn'
#    if datadir[-12:-11] == 'S': instr_id = 'gmoss'
#    
#    if not os.path.exists(path_procfits):
#        if os.path.exists(path_rawfits):
#            if len(os.listdir(path_rawfits)) > 0:
#                if os.path.exists(path_rawbias):
#                    if len(os.listdir(path_rawbias)) > 0:
#                        if instr_id != 'not recognized':
#                            if instr_id == 'gmosn':
#                                process_flatfield_files(path_rawfits,path_rawbias,path_procfits,instr_id)
#                            else:
#                                print('{:s} - GMOS-S reduction code not yet available.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
#                        else:
#                            print('{:s} - Instrument for {:s} not recognized.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
#                    else:
#                        print('{:s} - Bias directory in {:s} is empty.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
#                else:
#                    print('{:s} - Bias directory in {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
#            else:
#                print('{:s} - Flatfield directory in {:s} is empty.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
#        else:
#            print('{:s} - Flatfield directory in {:s} not found.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
#    else:
#        print('{:s} - Data in {:s} already processed.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))

print('Done.')




Directory /Users/hhsieh/Data/176P/ut20051229_gemini2_twiskyflat/ut20051229_geminiN_twiskyflat/procfits_twiskyflat/ successfully created.

2022-07-15 22:49:44 - Decompressing and splitting data files...
>>> Starting decompression of bz2 files in /Users/hhsieh/Data/176P/ut20051229_gemini2_twiskyflat/ut20051229_geminiN_twiskyflat/rawfits_twiskyflat/...
>>> Decompression of bz2 files complete.
>>> Starting decompression of fz files in /Users/hhsieh/Data/176P/ut20051229_gemini2_twiskyflat/ut20051229_geminiN_twiskyflat/rawfits_twiskyflat/...
>>> Decompression of fz files complete.
Removing N20051229S0329.fits...
Splitting N20051229S0338.fits...


IndexError: list index out of range